In [1]:
from datasets import load_dataset

dataset = load_dataset("empathetic_dialogues")  # or other dataset

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.15k [00:00<?, ?B/s]

empathetic_dialogues.py:   0%|          | 0.00/4.51k [00:00<?, ?B/s]

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token  # important for GPT2-based models

def preprocess_batch(batch):
    texts = [f"{c} {u}" for c, u in zip(batch['context'], batch['utterance'])]
    encodings = tokenizer(texts, truncation=True, padding="max_length", max_length=128)
    encodings["labels"] = encodings["input_ids"].copy()  # Needed for training loss
    return encodings

In [12]:
tokenized_dataset = dataset["train"].map(
    preprocess_batch,
    batched=True,
    remove_columns=dataset["train"].column_names
)

tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/76673 [00:00<?, ? examples/s]

In [13]:
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments

model = AutoModelForCausalLM.from_pretrained("distilgpt2")

In [14]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    save_steps=500,
    save_total_limit=1,
    logging_steps=100,
    report_to="none",  # No wandb or other logging
    fp16=True,  # Mixed precision for faster training (on GPU)
)

In [15]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

trainer.train()

<ipython-input-15-4ee6e095f1a9>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,1.390700
200,0.539800
300,0.525700
400,0.495600
500,0.489200
600,0.473200
700,0.487300
800,0.507100
900,0.478000
1000,0.490500


TrainOutput(global_step=28755, training_loss=0.4340769586511704, metrics={'train_runtime': 3544.9378, 'train_samples_per_second': 64.887, 'train_steps_per_second': 8.112, 'total_flos': 7512902159302656.0, 'train_loss': 0.4340769586511704, 'epoch': 3.0})

In [16]:
model.save_pretrained("empathetic-distilgpt2")
tokenizer.save_pretrained("empathetic-distilgpt2")

('empathetic-distilgpt2/tokenizer_config.json',
 'empathetic-distilgpt2/special_tokens_map.json',
 'empathetic-distilgpt2/vocab.json',
 'empathetic-distilgpt2/merges.txt',
 'empathetic-distilgpt2/added_tokens.json',
 'empathetic-distilgpt2/tokenizer.json')

In [17]:
%%writefile app.py
import streamlit as st
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

st.title("🧘‍♀️ Mental Health Support Chatbot")
st.markdown("Talk to me about anything on your mind. I'm here to help 💙")

tokenizer = AutoTokenizer.from_pretrained("empathetic-distilgpt2")
model = AutoModelForCausalLM.from_pretrained("empathetic-distilgpt2")

if "chat_history" not in st.session_state:
    st.session_state.chat_history = []

user_input = st.text_input("You:", "")

if st.button("Send") and user_input:
    input_ids = tokenizer.encode(user_input, return_tensors="pt")
    output = model.generate(input_ids, max_length=100, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(output[:, input_ids.shape[-1]:][0], skip_special_tokens=True)

    st.session_state.chat_history.append(("You", user_input))
    st.session_state.chat_history.append(("Bot", response))

for speaker, message in st.session_state.chat_history:
    st.write(f"**{speaker}**: {message}")

Writing app.py


In [ ]:
!pip install streamlit pyngrok

In [18]:

!streamlit run app.py &

from pyngrok import ngrok
print(ngrok.connect(8501))




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.214.235:8501

  Stopping...


ERROR:pyngrok.process.ngrok:t=2025-06-08T19:09:13+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-06-08T19:09:13+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"


PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.